# 코드 개요
1. 크롤링 사이트 : 네이버쇼핑 - 네이버 페이탭
2. 크롤링 데이터 : 상품명, url
3. 1페이지부터 20페이지까지 순회하며 url 및 상품명 수집

## 주요 모듈 import

In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from urllib.parse import quote
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

## depth2(계절가전), depth3(에어컨) 수집

In [2]:
cat_name = ['계절가전','에어컨']
    # 파일 저장에 사용
urls = ['https://search.shopping.naver.com/search/category/100000581','https://search.shopping.naver.com/search/category/100000620#']
    # depth2와 depth3의 페이지 링크 포함

In [ ]:
# 20페이지까지 상품명, url 수집
for i in range(2) :
            # range(2) 대신 range(len(urls)) 사용해도 무방 <= url 리스트 안에 있는 url을 순회하며 페이지 크롤링 해오는 게 목적인 반복문
    print(f"{'-'*20} {cat_name[i]} {'-'*20}")
            # 현재 어떤 depth에서 수집하고 있는지 정보 표시
    
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options) # 크롬드라이버 자동 인스톨, 안되면 직접 설치 필요
    #driver = webdriver.Chrome(service = Service('C:/Users/user/Desktop/작업용 폴더/chromedriver-win64/chromedriver-win64/chromedriver.exe'), options = chrome_options)
    driver.get(urls[i]) 
            # 계절가전, 에어컨 url 접근

    time.sleep(0.5)
    
    # 네이버 페이 클릭
    driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/ul/li[3]/a').click()
                                        # [네이버페이] 탭의 XPATH 값
    
    time.sleep(0.5)
    page_nums = 1
    prod_df = pd.DataFrame(columns = ['상품명','url'])
                # 상품명, url 두 열을 가지고 있는 데이터 프레임 완성
    while 1 :
            # while 1(True) : 무한 반복문 처리
        if page_nums == 21 :
            # 20페이지까지만 크롤링
            break
            
        SCROLL_PAUSE_SEC = 1

        # 스크롤 높이 가져옴
        last_height = driver.execute_script("return document.body.scrollHeight")

        while 1:
            # 끝까지 스크롤 다운 - 네이버 쇼핑 페이지가 스크롤을 내려야 추가정보를 가져오는 형태로 구성되어 있음
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # 1초 대기
            time.sleep(SCROLL_PAUSE_SEC)

            # 스크롤 다운 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
                    # 더이상 스크롤이 내려가지 않을때까지 해당 루틴 반복
            last_height = new_height
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        prod_list = soup.find_all('div',class_= 'product_title__Mmw2K')
                                                    # 상품명 요소만 추출
        for prod in prod_list :
            url = str(prod).split('href="')[1].split()[0]
            prod_nm = str(prod).split('title=')[1].split('>')[0].replace('"','')
            new_df = pd.DataFrame([[prod_nm,url]],columns = ['상품명','url'])
            prod_df = pd.concat([prod_df,new_df])

        
        page_nums += 1
        if page_nums == 21 :
            break
        elif page_nums-1 == 1 :
            driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a').click()
        else :
            driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a[2]').click()
        time.sleep(1)
        print('page_nums:', page_nums-1, '수집 개수:',len(prod_df))

    prod_df = prod_df.reset_index(drop = True)
    file_name = f'{cat_name[i]}(네이버페이,800개).csv'
    prod_df.to_csv(file_name, index = False, encoding = 'utf-8-sig')